In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load your data
data = pd.read_csv("./aos_data/merged_phuc_data.csv")  # Replace with your file
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ', 'IR']
X = data[features]
y = data['Label'].map({'sleep': 0, 'awake': 1})  # Convert labels to 0/1

# Normalize data (save min/max for ESP32)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Decision Tree
model = DecisionTreeClassifier(max_depth=4)  # Keep it simple for ESP32
model.fit(X_train, y_train)

# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")

# Export min/max values for normalization
for i, feature in enumerate(features):
    print(f"{feature}_min = {scaler.data_min_[i]};")
    print(f"{feature}_max = {scaler.data_max_[i]};")

# Manually inspect the tree (for translation to C)
from sklearn.tree import export_text
tree_rules = export_text(model, feature_names=features)
print(tree_rules)

Accuracy: 1.00
AccelX_min = 9.375694;
AccelX_max = 10.74997;
AccelY_min = -0.88346;
AccelY_max = 2.142811;
AccelZ_min = -1.563414;
AccelZ_max = 2.619257;
GyroX_min = -1.031476;
GyroX_max = 0.501216;
GyroY_min = -1.821804;
GyroY_max = 1.174034;
GyroZ_min = -0.252606;
GyroZ_max = 0.099657;
IR_min = 974.0;
IR_max = 133423.0;
|--- AccelY <= 0.59
|   |--- IR <= 0.15
|   |   |--- class: 1
|   |--- IR >  0.15
|   |   |--- class: 0
|--- AccelY >  0.59
|   |--- AccelZ <= 0.92
|   |   |--- AccelZ <= 0.82
|   |   |   |--- class: 1
|   |   |--- AccelZ >  0.82
|   |   |   |--- GyroZ <= 0.66
|   |   |   |   |--- class: 0
|   |   |   |--- GyroZ >  0.66
|   |   |   |   |--- class: 1
|   |--- AccelZ >  0.92
|   |   |--- IR <= 0.89
|   |   |   |--- class: 1
|   |   |--- IR >  0.89
|   |   |   |--- class: 0

